<a href="https://colab.research.google.com/github/Forger-888/Baysys-3-Stop-Watch/blob/main/T02_Basys3_StopWatch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T02: Stopwatch on Basys 3

## Objective:


Design and implement a **stopwatch** on the **Basys 3 FPGA** board using Verilog, making use of the **4-digit** **7-segment** display LEDs for time display. The stopwatch should have the following features:  
1. **Reset functionality**: Press a button (e.g., `btnR`) to restart the stopwatch.  
2. **Time Display**: Display the elapsed time in minutes and seconds on the 4-digit 7-segment display.  
3. **Update Frequency**: Update the display at a reasonable frequency (e.g., every 1 second).

## Instructions  

### FPGA Project  
1. Create a new Vivado project for the Basys 3 FPGA board.  
2. Write Verilog code to implement the stopwatch logic.  
3. Map the Verilog design to the Basys 3 board's resources via the basys3 [constraint file](./Demo_Code/basys3_template.xdc) (`.xdc`).   
4. Generate the bitstream file.
5. Program the FPGA board with the bitstream.  
6. Test and debug your stopwatch design on the FPGA board to ensure correct functionality.  
7. Document your design, including the Verilog code, functional diagrams, and challenges faced during implementation.  
8. **Demo Vivado Project**: [T02_Basys3_StopWatch](./Demo_Code/T02_Basys3_StopWatch)

## Simulation


- [Verify functionality using testbenches and waveform analysis.](Simulation/simT02_Basys3_StopWatch_SSD.ipynb)  




### Additional Challenge (*Optional*):

  
Add features like **pause/resume functionality** to the stopwatch design.  

## Submission:


*This is a group work. Only one member needs to submit the materials.*  
1. A self-contained `ipynb` file which:  
   - **Google Colab link** and **GitHub link**.  
   - **Verilog codes** (module, testbench, waveform, AI prompt, etc.).  
   - A **brief report** documenting the design, Python code (e.g., logic expressions, AI-generated code), challenges, and lessons learned;
2. A **demonstration video** (less than 1 minute, `.mp4` format).
* Submit to **edimension** and share with instructor GitHub: `pe8sutd`, `pe8sutd@gmail.com`.  
3. The report should report how AI is used in the design (if applicable). The guideline: [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/pe8sutd/Colab_DSL_Practice25/blob/main/G01_Used_of_AI_in_Design.ipynb)


---

### Deadline:  
Refer to the due date in the **edimension submission folder**.  

---

### Grading:  
Based on:  
- Functionality of the stopwatch.  
- Clarity of Verilog code.  
- Quality of documentation.  
*Bonus points for additional features or creative implementations.*  

---



## Demo Video  


The short [video](https://youtu.be/xX5csfU79kQ) demonstrating the stopwatch’s functionality on the Basys 3 board.
https://youtu.be/xX5csfU79kQ

## First working Prototype code

In [ ]:
module stopwatch(
    input clk,        // 100MHz system clock
    input btnR,       // Reset button
    output reg [6:0] seg,  // 7-segment display output
    output reg [3:0] an   // 7-segment anode control
);

    reg [5:0] seconds = 0;  // 0 to 59 seconds
    reg [6:0] minutes = 0;  // 0 to 99 minutes

    wire clk_1hz, clk_1khz;
    reg [1:0] count; // 2-bit counter for multiplexing

    // Instantiate clock dividers
    clk_divider #(100000000) clk_1hz_gen (.clk_in(clk), .reset(btnR), .clk_out(clk_1hz));   // 1Hz for time update
    clk_divider #(100000) clk_1khz_gen (.clk_in(clk), .reset(btnR), .clk_out(clk_1khz));   // 1kHz for multiplexing

    // Time Counter: Updates every second
    always @(posedge clk_1hz or posedge btnR) begin
        if (btnR) begin
            seconds <= 0;
            minutes <= 0;
        end
        else begin
            if (seconds == 59) begin
                seconds <= 0;
                if (minutes == 99)
                    minutes <= 0;
                else
                    minutes <= minutes + 1;
            end
            else begin
                seconds <= seconds + 1;
            end
        end
    end

    // 7-segment digit selection (multiplexing)
    always @(posedge clk_1khz) begin
        count <= count + 1;
    end

    // Extract digits for MM:SS display
    wire [3:0] min_tens = minutes / 10;
    wire [3:0] min_ones = minutes % 10;
    wire [3:0] sec_tens = seconds / 10;
    wire [3:0] sec_ones = seconds % 10;

    // 7-segment display multiplexing
    always @(*) begin
        case(count)
            2'b00: begin an = 4'b1110; seg = seg_map(sec_ones); end // L0 -> Seconds (ones)
            2'b01: begin an = 4'b1101; seg = seg_map(sec_tens); end // L1 -> Seconds (tens)
            2'b10: begin an = 4'b1011; seg = seg_map(min_ones); end // L2 -> Minutes (ones)
            2'b11: begin an = 4'b0111; seg = seg_map(min_tens); end // L3 -> Minutes (tens)
            default: begin an = 4'b1111; seg = 7'b1111111; end // All off
        endcase
    end

    // 7-segment lookup function
    function [6:0] seg_map;
        input [3:0] num;
        case(num)
            4'd0: seg_map = 7'b1000000;
            4'd1: seg_map = 7'b1111001;
            4'd2: seg_map = 7'b0100100;
            4'd3: seg_map = 7'b0110000;
            4'd4: seg_map = 7'b0011001;
            4'd5: seg_map = 7'b0010010;
            4'd6: seg_map = 7'b0000010;
            4'd7: seg_map = 7'b1111000;
            4'd8: seg_map = 7'b0000000;
            4'd9: seg_map = 7'b0010000;
            default: seg_map = 7'b1111111; // Blank display
        endcase
    endfunction

endmodule

// Clock Divider Module (Parameterized)
module clk_divider #(parameter DIV_VALUE = 100000000)(
    input clk_in,
    input reset,
    output reg clk_out
);
    reg [$clog2(DIV_VALUE)-1:0] count = 0;

    always @(posedge clk_in or posedge reset) begin
        if (reset) begin
            count <= 0;
            clk_out <= 0;
        end else if (count == DIV_VALUE/2 - 1) begin
            clk_out <= ~clk_out;
            count <= 0;
        end else begin
            count <= count + 1;
        end
    end
endmodule


# Stopwatch with Start/Stop

In [ ]:
module stopwatch(
    input clk,        // 100MHz system clock
    input btnR,       // Reset button
    input btnC,       // Start/Stop button
    output reg [6:0] seg,  // 7-segment display output
    output reg [3:0] an   // 7-segment anode control
);

    reg [5:0] seconds = 0;  // 0 to 59 seconds
    reg [6:0] minutes = 0;  // 0 to 99 minutes
    reg state = 1'b0; // Stopwatch state start = 1/ stop = 0

    wire clk_1hz, clk_1khz;
    reg [1:0] count; // 2-bit counter for multiplexing

    wire debounced_btnC;
    debounce debounce_btnC (.clk(clk), .reset(btnR), .button_in(btnC), .button_out(debounced_btnC));

    // Instantiate clock dividers
    clk_divider #(100000000) clk_1hz_gen (.clk_in(clk), .reset(btnR), .clk_out(clk_1hz));   // 1Hz for time update
    clk_divider #(100000) clk_1khz_gen (.clk_in(clk), .reset(btnR), .clk_out(clk_1khz));   // 1kHz for multiplexing

    // **Asynchronous Start/Stop/Reset Control**
    always @(posedge debounced_btnC or posedge btnR) begin
        if (btnR)
            state <= 0; // Reset to zero
        else
            state <= ~state;  // Toggle State for Start/Pause
    end

    // **Time Counter: Updates every second**
    always @(posedge clk_1hz or posedge btnR) begin
        if (btnR) begin
            seconds <= 0;
            minutes <= 0;
        end
        else if (state) begin
            if (seconds == 59) begin
                seconds <= 0;
                if (minutes == 99)
                    minutes <= 0;
                else
                    minutes <= minutes + 1;
            end
            else begin
                seconds <= seconds + 1;
            end
        end
    end

    // 7-segment digit selection (multiplexing)
    always @(posedge clk_1khz) begin
        count <= count + 1;
    end

    // Extract digits for MM:SS display
    wire [3:0] min_tens = minutes / 10;
    wire [3:0] min_ones = minutes % 10;
    wire [3:0] sec_tens = seconds / 10;
    wire [3:0] sec_ones = seconds % 10;

    // 7-segment display multiplexing
    always @(*) begin
        case(count)
            2'b00: begin an = 4'b1110; seg = seg_map(sec_ones); end // L0 -> Seconds (ones)
            2'b01: begin an = 4'b1101; seg = seg_map(sec_tens); end // L1 -> Seconds (tens)
            2'b10: begin an = 4'b1011; seg = seg_map(min_ones); end // L2 -> Minutes (ones)
            2'b11: begin an = 4'b0111; seg = seg_map(min_tens); end // L3 -> Minutes (tens)
            default: begin an = 4'b1111; seg = 7'b1111111; end // All off
        endcase
    end

    // 7-segment lookup function
    function [6:0] seg_map;
        input [3:0] num;
        case(num)
            4'd0: seg_map = 7'b1000000;
            4'd1: seg_map = 7'b1111001;
            4'd2: seg_map = 7'b0100100;
            4'd3: seg_map = 7'b0110000;
            4'd4: seg_map = 7'b0011001;
            4'd5: seg_map = 7'b0010010;
            4'd6: seg_map = 7'b0000010;
            4'd7: seg_map = 7'b1111000;
            4'd8: seg_map = 7'b0000000;
            4'd9: seg_map = 7'b0010000;
            default: seg_map = 7'b1111111; // Blank display
        endcase
    endfunction

endmodule

// Clock Divider Module (Parameterized)
module clk_divider #(parameter DIV_VALUE = 100000000)(
    input clk_in,
    input reset,
    output reg clk_out
);
    reg [$clog2(DIV_VALUE)-1:0] count = 0;

    always @(posedge clk_in or posedge reset) begin
        if (reset) begin
            count <= 0;
            clk_out <= 0;
        end else if (count == DIV_VALUE/2 - 1) begin
            clk_out <= ~clk_out;
            count <= 0;
        end else begin
            count <= count + 1;
        end
    end
endmodule

// **Debounce Module for Button**
module debounce (
    input wire clk,        // System clock (e.g., 100 MHz on Basys 3)
    input wire reset,      // Reset signal
    input wire button_in,  // Raw button input
    output reg button_out  // Debounced button output
);

    reg [21:0] count;      // 22-bit counter for debouncing
    reg button_prev;        // Previous state of the button

    always @(posedge clk or posedge reset) begin
        if (reset) begin
            count <= 20'b0;
            button_prev <= 1'b0;
            button_out <= 1'b0;
        end else begin
            button_prev <= button_in;
            if (button_prev != button_in) begin
                count <= 20'b0;
            end else if (count == 20'hFFFFF) begin
                button_out <= button_prev;
            end else begin
                count <= count + 1;
            end
        end
    end

endmodule


V1 working

In [ ]:
module stopwatch(
    input clk,        // 100MHz system clock
    input btnR,       // Reset button
    input btnC,       // Start/Stop button
    output reg [6:0] seg,  // 7-segment display output
    output reg [3:0] an   // 7-segment anode control
);

    reg [5:0] seconds = 0;  // 0 to 59 seconds
    reg [6:0] minutes = 0;  // 0 to 99 minutes
    reg state = 1'b1; // Stopwatch state start = 1/ stop = 0

    wire clk_1hz, clk_1khz;
    reg [1:0] count; // 2-bit counter for multiplexing

    wire debounced_btnC;
     debounce debounce_btnC (.clk(clk), .reset(btnR), .button_in(btnC), .button_out(debounced_btnC));

    // Instantiate clock dividers
    clk_divider #(100000000) clk_1hz_gen (.clk_in(clk), .reset(btnR), .clk_out(clk_1hz));   // 1Hz for time update
    clk_divider #(100000) clk_1khz_gen (.clk_in(clk), .reset(btnR), .clk_out(clk_1khz));   // 1kHz for multiplexing

    // Start/Stop/Reset
    always @(posedge clk) begin
        if (btnR)
            state <= 0; //Reset to zero
        else if (debounced_btnC)
            state <= ~state;  //Toggle State for Start/Pause
    end


    // Time Counter: Updates every second
    always @(posedge clk_1hz or posedge btnR) begin
        if (btnR) begin
            seconds <= 0;
            minutes <= 0;
        end
        else if (state) begin
            if (seconds == 59) begin
                seconds <= 0;
                if (minutes == 99)
                    minutes <= 0;
                else
                    minutes <= minutes + 1;
            end
            else begin
                seconds <= seconds + 1;
            end
        end
    end

    // 7-segment digit selection (multiplexing)
    always @(posedge clk_1khz) begin
        count <= count + 1;
    end

    // Extract digits for MM:SS display
    wire [3:0] min_tens = minutes / 10;
    wire [3:0] min_ones = minutes % 10;
    wire [3:0] sec_tens = seconds / 10;
    wire [3:0] sec_ones = seconds % 10;

    // 7-segment display multiplexing
    always @(*) begin
        case(count)
            2'b00: begin an = 4'b1110; seg = seg_map(sec_ones); end // L0 -> Seconds (ones)
            2'b01: begin an = 4'b1101; seg = seg_map(sec_tens); end // L1 -> Seconds (tens)
            2'b10: begin an = 4'b1011; seg = seg_map(min_ones); end // L2 -> Minutes (ones)
            2'b11: begin an = 4'b0111; seg = seg_map(min_tens); end // L3 -> Minutes (tens)
            default: begin an = 4'b1111; seg = 7'b1111111; end // All off
        endcase
    end

    // 7-segment lookup function
    function [6:0] seg_map;
        input [3:0] num;
        case(num)
            4'd0: seg_map = 7'b1000000;
            4'd1: seg_map = 7'b1111001;
            4'd2: seg_map = 7'b0100100;
            4'd3: seg_map = 7'b0110000;
            4'd4: seg_map = 7'b0011001;
            4'd5: seg_map = 7'b0010010;
            4'd6: seg_map = 7'b0000010;
            4'd7: seg_map = 7'b1111000;
            4'd8: seg_map = 7'b0000000;
            4'd9: seg_map = 7'b0010000;
            default: seg_map = 7'b1111111; // Blank display
        endcase
    endfunction

endmodule

// Clock Divider Module (Parameterized)
module clk_divider #(parameter DIV_VALUE = 100000000)(
    input clk_in,
    input reset,
    output reg clk_out
);
    reg [$clog2(DIV_VALUE)-1:0] count = 0;

    always @(posedge clk_in or posedge reset) begin
        if (reset) begin
            count <= 0;
            clk_out <= 0;
        end else if (count == DIV_VALUE/2 - 1) begin
            clk_out <= ~clk_out;
            count <= 0;
        end else begin
            count <= count + 1;
        end
    end
endmodule

// **Debounce Module for Button**
module debounce (
    input wire clk,        // System clock (e.g., 100 MHz on Basys 3)
    input wire reset,      // Reset signal
    input wire button_in,  // Raw button input
    output reg button_out  // Debounced button output
);

    reg [21:0] count;      // 22-bit counter for debouncing
    reg button_prev;        // Previous state of the button

    always @(posedge clk or posedge reset) begin
        if (reset) begin
            count <= 20'b0;
            button_prev <= 1'b0;
            button_out <= 1'b0;
        end else begin
            button_prev <= button_in;
            if (button_prev != button_in) begin
                count <= 20'b0;
            end else if (count == 20'hFFFFF) begin
                button_out <= button_prev;
            end else begin
                count <= count + 1;
            end
        end
    end

endmodule

Not Working

In [ ]:
module stopwatch(
    input clk,        // 100MHz system clock
    input btnR,       // Reset button
    input btnC,       // Start/Pause button
    output reg [6:0] seg,  // 7-segment display output
    output reg [3:0] an,   // 7-segment anode control
    output reg [4:0] led   // 5 LEDs for debugging
);

    reg [5:0] seconds = 0;  // 0 to 59 seconds
    reg [6:0] minutes = 0;  // 0 to 99 minutes
    reg running = 0;

    wire clk_1hz, clk_1khz;
    reg [1:0] count;

    // Instantiate clock dividers
    clk_divider #(100000000) clk_1hz_gen (.clk_in(clk), .reset(btnR), .clk_out(clk_1hz));
    clk_divider #(100000) clk_1khz_gen (.clk_in(clk), .reset(btnR), .clk_out(clk_1khz));

    // **Debounce & Edge Detection for btnC**
    reg [1:0] btnC_stable = 2'b00;  // ✅ Use 2-bit register for stability
    reg btnC_prev = 0;
    wire btnC_edge;

    always @(posedge clk) begin
        btnC_stable <= {btnC_stable[0], btnC};  // ✅ Shift Register for button stability
        btnC_prev <= btnC_stable[1];
    end

    assign btnC_edge = btnC_stable[1] & ~btnC_prev;  // ✅ Detect rising edge

    // **Start/Pause Toggle**
    always @(posedge clk) begin
        if (btnC_edge) begin
            running <= ~running;
        end
    end

    // **Fix: Proper Synchronization of `running` into `clk_1hz`**
    reg running_sync1 = 0, running_sync2 = 0, toggle_ff = 0;

    always @(posedge clk) begin
        if (btnC_edge) begin
            toggle_ff <= ~toggle_ff; // ✅ Toggle on button press
        end
    end

    always @(posedge clk_1hz) begin
        running_sync1 <= toggle_ff;  // ✅ First stage sync
        running_sync2 <= running_sync1;  // ✅ Second stage sync
    end

    // **Time Counter**
    always @(posedge clk_1hz or posedge btnR) begin
        if (btnR) begin
            seconds <= 0;
            minutes <= 0;
            running_sync1 <= 0;
            running_sync2 <= 0;
        end
        else if (running_sync2) begin
            if (seconds == 59) begin
                seconds <= 0;
                if (minutes == 99)
                    minutes <= 0;
                else
                    minutes <= minutes + 1;
            end
            else begin
                seconds <= seconds + 1;
            end
        end
    end

    // **Debug LEDs**
    always @(posedge clk) begin
        led[0] <= clk_1hz;      // ✅ LED 0 blinks every second (Clock Debug)
        led[1] <= running;      // ✅ LED 1 toggles ON/OFF when pressing `btnC`
        led[2] <= btnC_stable[1];  // ✅ LED 2 lights up when `btnC` is pressed (Debounce Debug)
        led[3] <= running_sync2; // ✅ LED 3 now follows `running_sync2`
        led[4] <= seconds[0];   // ✅ LED 4 blinks every 2 sec if counter is updating
    end

    // **7-segment digit selection (multiplexing)**
    always @(posedge clk_1khz) begin
        count <= count + 1;
    end

    // **Extract digits for MM:SS display**
    wire [3:0] min_tens = minutes / 10;
    wire [3:0] min_ones = minutes % 10;
    wire [3:0] sec_tens = seconds / 10;
    wire [3:0] sec_ones = seconds % 10;

    // **7-segment display multiplexing**
    always @(*) begin
        case(count)
            2'b00: begin an = 4'b1110; seg = seg_map(sec_ones); end // L0 -> Seconds (ones)
            2'b01: begin an = 4'b1101; seg = seg_map(sec_tens); end // L1 -> Seconds (tens)
            2'b10: begin an = 4'b1011; seg = seg_map(min_ones); end // L2 -> Minutes (ones)
            2'b11: begin an = 4'b0111; seg = seg_map(min_tens); end // L3 -> Minutes (tens)
            default: begin an = 4'b1111; seg = 7'b1111111; end // All off
        endcase
    end

    // **7-Segment Decoder Function**
    function [6:0] seg_map;
        input [3:0] digit;
        begin
            case(digit)
                4'd0: seg_map = 7'b1000000; // 0
                4'd1: seg_map = 7'b1111001; // 1
                4'd2: seg_map = 7'b0100100; // 2
                4'd3: seg_map = 7'b0110000; // 3
                4'd4: seg_map = 7'b0011001; // 4
                4'd5: seg_map = 7'b0010010; // 5
                4'd6: seg_map = 7'b0000010; // 6
                4'd7: seg_map = 7'b1111000; // 7
                4'd8: seg_map = 7'b0000000; // 8
                4'd9: seg_map = 7'b0010000; // 9
                default: seg_map = 7'b1111111; // Blank display
            endcase
        end
    endfunction

endmodule

// **Clock Divider Module (Parameterized)**
module clk_divider #(parameter DIV_VALUE = 100000000)(
    input clk_in,
    input reset,
    output reg clk_out
);
    reg [$clog2(DIV_VALUE)-1:0] count = 0;

    always @(posedge clk_in or posedge reset) begin
        if (reset) begin
            count <= 0;
            clk_out <= 0;
        end else if (count == DIV_VALUE/2 - 1) begin
            clk_out <= ~clk_out;
            count <= 0;
        end else begin
            count <= count + 1;
        end
    end
endmodule


In [ ]:
module stopwatch(
    input clk,        // 100MHz system clock
    input btnR,       // Reset button
    input btnC,       // Start/Pause button
    output reg [6:0] seg,  // 7-segment display output
    output reg [3:0] an   // 7-segment anode control
);

    reg [5:0] seconds = 0;  // 0 to 59 seconds
    reg [6:0] minutes = 0;  // 0 to 99 minutes

    wire clk_1hz, clk_1khz;
    reg [1:0] count; // 2-bit counter for multiplexing

    reg state = 0; // 0: stop/pause, 1: start
    wire debounced_btnC;

    // Instantiate clock dividers
    clk_divider #(100000000) clk_1hz_gen (.clk_in(clk), .reset(btnR), .clk_out(clk_1hz));   // 1Hz for time update
    clk_divider #(100000) clk_1khz_gen (.clk_in(clk), .reset(btnR), .clk_out(clk_1khz));   // 1kHz for multiplexing

    // Debounce the start/stop button
    debounce debounce_btnC (.clk(clk), .reset(btnR), .button_in(btnC), .button_out(debounced_btnC));

    // Button edge detection
    reg btnC_prev = 0;
    always @(posedge clk) begin
        btnC_prev <= debounced_btnC;
    end
    wire btnC_edge = debounced_btnC & ~btnC_prev; // Detect rising edge

    // Toggle state on button press
    always @(posedge clk) begin
        if (btnC_edge) begin
            state <= ~state;
        end
    end

    // **Fixed: Time Counter - Holds value when paused**
    always @(posedge clk_1hz or posedge btnR) begin
        if (btnR) begin
            seconds <= 0;
            minutes <= 0;
            state <= 0; // Reset also stops the timer
        end
        else if (state == 1) begin  // Only update if state == 1 (running)
            if (seconds == 59) begin
                seconds <= 0;
                if (minutes == 99)
                    minutes <= 0;
                else
                    minutes <= minutes + 1;
            end
            else begin
                seconds <= seconds + 1;
            end
        end
    end

    // 7-segment digit selection (multiplexing)
    always @(posedge clk_1khz) begin
        count <= count + 1;
    end

    // Extract digits for MM:SS display
    wire [3:0] min_tens = minutes / 10;
    wire [3:0] min_ones = minutes % 10;
    wire [3:0] sec_tens = seconds / 10;
    wire [3:0] sec_ones = seconds % 10;

    // 7-segment display multiplexing
    always @(*) begin
        case(count)
            2'b00: begin an = 4'b1110; seg = seg_map(sec_ones); end // L0 -> Seconds (ones)
            2'b01: begin an = 4'b1101; seg = seg_map(sec_tens); end // L1 -> Seconds (tens)
            2'b10: begin an = 4'b1011; seg = seg_map(min_ones); end // L2 -> Minutes (ones)
            2'b11: begin an = 4'b0111; seg = seg_map(min_tens); end // L3 -> Minutes (tens)
            default: begin an = 4'b1111; seg = 7'b1111111; end // All off
        endcase
    end

    // 7-segment lookup function
    function [6:0] seg_map;
        input [3:0] num;
        case(num)
            4'd0: seg_map = 7'b1000000;
            4'd1: seg_map = 7'b1111001;
            4'd2: seg_map = 7'b0100100;
            4'd3: seg_map = 7'b0110000;
            4'd4: seg_map = 7'b0011001;
            4'd5: seg_map = 7'b0010010;
            4'd6: seg_map = 7'b0000010;
            4'd7: seg_map = 7'b1111000;
            4'd8: seg_map = 7'b0000000;
            4'd9: seg_map = 7'b0010000;
            default: seg_map = 7'b1111111; // Blank display
        endcase
    endfunction

endmodule

// **Debounce Module for Button**
module debounce (
    input wire clk,        // System clock (e.g., 100 MHz on Basys 3)
    input wire reset,      // Reset signal
    input wire button_in,  // Raw button input
    output reg button_out  // Debounced button output
);

    reg [19:0] count;      // 20-bit counter for debouncing
    reg button_prev;        // Previous state of the button

    always @(posedge clk or posedge reset) begin
        if (reset) begin
            count <= 20'b0;
            button_prev <= 1'b0;
            button_out <= 1'b0;
        end else begin
            button_prev <= button_in;
            if (button_prev != button_in) begin
                count <= 20'b0;
            end else if (count == 20'hFFFFF) begin
                button_out <= button_prev;
            end else begin
                count <= count + 1;
            end
        end
    end

endmodule

// **Clock Divider Module (Parameterized)**
module clk_divider #(parameter DIV_VALUE = 100000000)(
    input clk_in,
    input reset,
    output reg clk_out
);
    reg [$clog2(DIV_VALUE)-1:0] count = 0;

    always @(posedge clk_in or posedge reset) begin
        if (reset) begin
            count <= 0;
            clk_out <= 0;
        end else if (count == DIV_VALUE/2 - 1) begin
            clk_out <= ~clk_out;
            count <= 0;
        end else begin
            count <= count + 1;
        end
    end
endmodule
